In [182]:
import pandas as pd
import jieba
testjd = pd.read_excel('D:\泰迪杯\c题\C题测试数据\酒店评论（测试数据）.xlsx')
jdscore = pd.read_excel('D:/泰迪杯/c题/附件2/酒店评分.xlsx')
testjd

,酒店名称,评论日期,评论内容,入住房型
0,H51,2020-01-02,前台服务态度非常好，很尽责 讲的很详细 房间还有水果糖水 很棒,典雅园林大床房
1,H51,2020-01-02,温泉很舒服、大堂很气派 前台服务态度非常非常非常非常好,典雅园林大床房
2,H51,2020-01-02,环境优美，服务人员很热情 、,豪华园林大床房
3,H51,2020-01-04,环境很好 服务态度好 房间很好 ！,典雅园林双床房
4,H51,2020-01-04,住的环境还行，唯一不好的是洗脸盆，有黄印子在上面，看起来有点恶心，游泳池没开放，热矿泉温度个...,猪猪侠酷芽亲子家庭房
...,...,...,...,...
3031,H60,2020-08-23,酒店干净舒适，交通算方便，下次去还会去住,贵宾楼特惠大床房
3032,H60,2020-08-23,因為一早要在拱北醫院檢查，臨時訂的。感覺還是可以的。價格也算公道，也不需要去挑剔過多的。,主楼豪华大床房
3033,H60,2020-08-23,不错，还好吧,贵宾楼贵宾大床房
3034,H60,2020-08-23,房间太小了,主楼特价大床房


In [186]:
df = testjd['评论内容']
df.duplicated()
df.drop_duplicates()

0                         前台服务态度非常好，很尽责 讲的很详细 房间还有水果糖水 很棒
1                             温泉很舒服、大堂很气派 前台服务态度非常非常非常非常好
2                                          环境优美，服务人员很热情 、
3                                       环境很好 服务态度好 房间很好 ！
4       住的环境还行，唯一不好的是洗脸盆，有黄印子在上面，看起来有点恶心，游泳池没开放，热矿泉温度个...
                              ...                        
2734                                 酒店干净舒适，交通算方便，下次去还会去住
2735         因為一早要在拱北醫院檢查，臨時訂的。感覺還是可以的。價格也算公道，也不需要去挑剔過多的。
2736                                               不错，还好吧
2737                                                房间太小了
2738                                     早餐品种不多，硬件不错，服务很好
Name: 评论内容, Length: 2633, dtype: object

In [187]:
testjd.index=testjd['酒店名称']
x = [f'H{x}{y}' for x in range(5,7) for y in range(0,10)][1:11]
df = [testjd.loc[f'{x}']['评论内容'] for x in x]

In [188]:
df=[str(i) for i in df]

In [189]:
import re
#加载停用词
stoplist= [word.strip() for word in open('stopwords.txt',encoding='gb2312').readlines()]
# print(stoplist)
sentences_list = df
# 对句子进行分词
def seg_depart(sentence):
    # 去掉非汉字字符
    sentence = re.sub(r'[^\u4e00-\u9fa5]+','',sentence)
    sentence_depart = jieba.cut(sentence.strip())
    word_list = []
    for word in sentence_depart:
        if word not in stoplist:
            word_list.append(word)
    # 如果句子整个被过滤掉了，如：'02-2717:56'被过滤，那就返回[],保持句子的数量不变
    return word_list

sentence_word_list = []
for sentence in sentences_list:
    line_seg = seg_depart(sentence)
    sentence_word_list.append(line_seg)

df = sentences_list

In [190]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
 
#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfVectorizer()

df = tf_idf_transformer.fit_transform(df)
type(df)

scipy.sparse.csr.csr_matrix

In [191]:
from sklearn.externals import joblib
y_test = jdscore.sample(n=10,axis=0)

In [400]:
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor(n_estimators=27,max_samples=1.0,max_features=1.0,n_jobs=None)
joblib.dump(model_BaggingRegressor,'B1_model')

['B1_model']

In [401]:
model = joblib.load('B1_model')#加载模型

In [402]:
y_test1 = y_test['总得分']
model.fit(df,y_test1)
y_hat1 = model.predict(df)
print("总得分得分评分：",y_hat1)

总得分得分评分： [4.77407407 4.6962963  4.70740741 4.77407407 4.75555556 4.62222222
 4.63333333 4.75925926 4.69259259 4.71481481]


In [403]:
model.score(df,y_test1)

0.8762982559278827

In [404]:
y_test2 = y_test['服务得分']
model.fit(df,y_test2)
y_hat2 = model.predict(df)
print("服务得分：",y_hat2)

服务得分： [4.77037037 4.7        4.68888889 4.76296296 4.80740741 4.64814815
 4.63333333 4.6962963  4.7        4.70740741]


In [405]:
y_test3 = y_test['位置得分']
model.fit(df,y_test3)
y_hat3 = model.predict(df)
print("位置得分：",y_hat3)

位置得分： [4.79259259 4.78518519 4.78888889 4.78148148 4.78518519 4.73703704
 4.72592593 4.78888889 4.78518519 4.73703704]


In [406]:
y_test4 = y_test['设施得分']
model.fit(df,y_test4)
y_hat4 = model.predict(df)
print("设施得分：",y_hat4)

设施得分： [4.73703704 4.61851852 4.64444444 4.73333333 4.73333333 4.58148148
 4.6        4.79259259 4.70740741 4.67037037]


In [407]:
y_test5 = y_test['卫生得分']
model.fit(df,y_test5)
y_hat5 = model.predict(df)
print("卫生得分：",y_hat5)

卫生得分： [4.85925926 4.73703704 4.74444444 4.82222222 4.84074074 4.74444444
 4.74074074 4.85555556 4.78888889 4.64074074]


In [408]:
y_test6 = y_test['性价比得分']
model.fit(df,y_test6)
y_hat6 = model.predict(df)
print("性价比得分：",y_hat6)

性价比得分： [4.02962963 4.12222222 4.         4.01481481 4.08148148 4.
 4.01481481 4.         4.03703704 4.13333333]


In [413]:
x = [f'H{x}{y}' for x in range(5,7) for y in range(0,10)][1:11]

In [414]:
import pandas as pd
dataset = pd.DataFrame({'酒店名称':x,'总得分': y_hat1, '服务得分': y_hat2, '位置得分': y_hat3, '设施得分': y_hat4, '卫生得分': y_hat5, '性价比得分': y_hat6,})

In [415]:
dataset.set_index(['酒店名称'],inplace=True)
dataset

,总得分,服务得分,位置得分,设施得分,卫生得分,性价比得分
酒店名称,,,,,,
H51,4.774074,4.770370,4.792593,4.737037,4.859259,4.029630
H52,4.696296,4.700000,4.785185,4.618519,4.737037,4.122222
H53,4.707407,4.688889,4.788889,4.644444,4.744444,4.000000
H54,4.774074,4.762963,4.781481,4.733333,4.822222,4.014815
H55,4.755556,4.807407,4.785185,4.733333,4.840741,4.081481
H56,4.622222,4.648148,4.737037,4.581481,4.744444,4.000000
H57,4.633333,4.633333,4.725926,4.600000,4.740741,4.014815
H58,4.759259,4.696296,4.788889,4.792593,4.855556,4.000000
H59,4.692593,4.700000,4.785185,4.707407,4.788889,4.037037


In [416]:
dataset.to_excel('酒店评分.xlsx')